In [1]:
import os
import sys
import pandas as pd
import numpy as np
from ast import literal_eval

os.getcwd()

'/Users/chris/GitHub/mathai/src'

In [68]:
from main import print_problems_df
    #map_course_files, parse_course_files, parse_problem_sets, 

Loaded dataframes: 
worksheet_files_df:
 index name:  None 
columns:  Index(['Unnamed: 0', 'unit', 'file_count', 'filename'], dtype='object') 
149 rows:  149 

problem_sets_df:
 index name:  problem_set_ID 
columns:  Index(['filename', 'head', 'body', 'problems_list', 'problem_count',
       'problem_IDs'],
      dtype='object') 
149 rows:  149 

problems_df:
 index name:  problem_ID 
columns:  Index(['problem_set_ID', 'question'], dtype='object') 
1855 rows:  1855 

standards_df:
 index name:  None 
columns:  Index(['course', 'chapter', 'topic', 'ccss_ID'], dtype='object') 
314 rows:  314 

standards_desc_df:
 index name:  ccss_ID 
columns:  Index(['description'], dtype='object') 
131 rows:  131 



## problems and problem set dataframes, refactoring

In [90]:
newpage_df.to_csv('/Users/chris/GitHub/mathai/db/newpage_df.csv')
problem_df.to_csv('/Users/chris/GitHub/mathai/db/problem_df.csv')
pset_df.to_csv('/Users/chris/GitHub/mathai/db/pset_df.csv')
multicol_error_pset_df.to_csv('/Users/chris/GitHub/mathai/db/multicol_error_pset_df.csv')

In [2]:
problem_df = pd.read_csv('../db/problem_df.csv', index_col='problem_ID')
newpage_df = pd.read_csv('../db/newpage_df.csv', index_col='problem_ID')
cols_into_lists = {'problem_IDs':literal_eval, 'head':literal_eval, 'body':literal_eval, 'problems_list':literal_eval}
pset_df = pd.read_csv('../db/pset_df.csv', index_col='problem_set_ID', converters=cols_into_lists)
multicol_error_pset_df = pd.read_csv('../db/multicol_error_pset_df.csv', index_col='problem_set_ID', converters=cols_into_lists)
print('problem_df: ', len(problem_df), problem_df.index.name, problem_df.columns)
print('newpage_df: ', len(newpage_df), newpage_df.index.name, newpage_df.columns)
print('pset_df: ', len(pset_df), pset_df.index.name, pset_df.columns)
print('multicol_error_pset_df: ', len(multicol_error_pset_df))

problem_df:  2107 problem_ID Index(['problem_set_ID', 'question', 'q_len', 'file'], dtype='object')
newpage_df:  513 problem_ID Index(['problem_set_ID', 'question', 'q_len', 'file'], dtype='object')
pset_df:  172 problem_set_ID Index(['filename', 'head', 'body', 'problems_list', 'problem_count', 'path',
       'unit', 'file'],
      dtype='object')
multicol_error_pset_df:  38


In [8]:
pset_df

,filename,head,body,problems_list,problem_count,path,unit,file
problem_set_ID,,,,,,,,
0,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[\begin{document}\n, \subsubsection*{1-1 Class...","[\n, \item Given the line segment $\overline...",[Given the line segment $\overline{PQ}$ shown ...,9,/Users/chris/GitHub/course-files/Geom,01-Intro,1-1CW_Measurement.tex
1,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[\begin{document}\n, \subsubsection*{1-1 Do No...","[\n, \n, \item In the following two problems...","[\n\n, In the following two problems, solve fo...",8,/Users/chris/GitHub/course-files/Geom,01-Intro,1-1DN_Algebra.tex
2,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[\begin{document}\n, \subsubsection*{1-1 Homew...","[\n, \n, \item In the following two problems...","[\n\n, In the following two problems, solve fo...",9,/Users/chris/GitHub/course-files/Geom,01-Intro,1-1HW_Algebra.tex
3,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[\begin{document}\n, \subsubsection*{1-3 Do No...","[ \item I have a compass, ruler, protractor, ...","[I have a compass, ruler, protractor, notebook...",6,/Users/chris/GitHub/course-files/Geom,01-Intro,1-3DN_Function-intercepts.tex
4,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[\begin{document}\n, \subsubsection*{Homework ...","[\n, \item Construct an equilateral triang...",[Construct an equilateral triangle with $\over...,11,/Users/chris/GitHub/course-files/Geom,01-Intro,1-3HW_Equilateral-triangle.tex
...,...,...,...,...,...,...,...,...
167,/Users/chris/GitHub/course-files/Geom/10-Trig+...,"[\begin{document}\n, \subsubsection*{10.3 Do N...","[\n, \n, \item Write down the slope perpendi...","[\n\n, Write down the slope perpendicular to t...",10,/Users/chris/GitHub/course-files/Geom,10-Trig+similarity+analytics,10-2DN_Analytics-review.tex
168,/Users/chris/GitHub/course-files/Geom/10-Trig+...,"[\begin{document}\n, \subsubsection*{10.3b Do ...","[\n, \item Write down the slope parallel or ...",[Write down the slope parallel or perpendicula...,13,/Users/chris/GitHub/course-files/Geom,10-Trig+similarity+analytics,10-2bDN_Analytics-review.tex
169,/Users/chris/GitHub/course-files/Geom/10-Trig+...,"[\begin{document}\n, \subsubsection*{10.4 Do N...","[\n, \item Write down the slope perpendicula...",[Write down the slope perpendicular to the giv...,8,/Users/chris/GitHub/course-files/Geom,10-Trig+similarity+analytics,10-4DN_Tangent-situations.tex


In [3]:
p_df['file'] = problem_df.problem_set_ID.apply(lambda x: pset_df.loc[x, 'file'])
p_df['q_len'] = problem_df.question.str.len()
p_df.sort_values('q_len', inplace=True)
p_df

NameError: name 'p_df' is not defined

In [63]:
#print(pset_df.loc[62, 'body'])
multicol_error_pset_df = pset_df[pset_df.body.apply(lambda x: r'[\\item' in str(x))].copy()

In [94]:
new_cols = ['unit', 'file', 'problem_count', 'head', 'body', 'problems_list', 'path', 'filename']
pset_df = pset_df.reindex(columns=new_cols)
pset_df[pset_df.unit.str.startswith('01')]

,unit,file,problem_count,head,body,problems_list,path,filename
problem_set_ID,,,,,,,,
0,01-Intro,1-1CW_Measurement.tex,9,"[\begin{document}\n, \subsubsection*{1-1 Class...","[\n, \item Given the line segment $\overline...",[Given the line segment $\overline{PQ}$ shown ...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/01-Intro...
1,01-Intro,1-1DN_Algebra.tex,8,"[\begin{document}\n, \subsubsection*{1-1 Do No...","[\n, \n, \item In the following two problems...","[\n\n, In the following two problems, solve fo...",/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/01-Intro...
2,01-Intro,1-1HW_Algebra.tex,9,"[\begin{document}\n, \subsubsection*{1-1 Homew...","[\n, \n, \item In the following two problems...","[\n\n, In the following two problems, solve fo...",/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/01-Intro...
3,01-Intro,1-3DN_Function-intercepts.tex,6,"[\begin{document}\n, \subsubsection*{1-3 Do No...","[ \item I have a compass, ruler, protractor, ...","[I have a compass, ruler, protractor, notebook...",/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/01-Intro...
4,01-Intro,1-3HW_Equilateral-triangle.tex,11,"[\begin{document}\n, \subsubsection*{Homework ...","[\n, \item Construct an equilateral triang...",[Construct an equilateral triangle with $\over...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/01-Intro...
5,01-Intro,1-4HW_Segment-addition.tex,11,"[\begin{document}\n, \subsubsection*{Homework ...","[ \item I have a compass, ruler, protractor...","[I have a compass, ruler, protractor, notebook...",/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/01-Intro...
6,01-Intro,1-5CW_Angle-terminology.tex,8,"[\begin{document}\n, \subsubsection*{Classwork...","[ \item I have a compass, ruler, protractor...","[I have a compass, ruler, protractor, notebook...",/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/01-Intro...
7,01-Intro,1-5HW_Pretest-segments-intro.tex,18,"[\begin{document}\n, \subsubsection*{1-8 Homew...",[ \item Points that are all located on the ...,[Points that are all located on the same line ...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/01-Intro...
8,01-Intro,1-6DN_Angle-pairs.tex,6,"[\begin{document}\n, \subsubsection*{1.6 Do No...","[\n, \item Complete the construction of an e...",[Complete the construction of an equilateral t...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/01-Intro...


In [133]:
print(pset_df.loc[62, 'problems_list'][7])
pset_df.loc[62, 'problems_list'][8]

Given $\triangle ABP \sim \triangle JKP$ as shown below. $AP=5.7$, $JP=11.4$, and $JK=14.8$. Find $AB$.
 \begin{flushright}
   \begin{tikzpicture}[scale=1.4]
       \draw [thick]
         (0.25,-1)node[right]{$B$}--
         (-0.5,2)node[left]{$K$}--
         (4,0)node[right]{$J$}--
         (0,0)node[above right]{$P$}--
         (-2,0)node[left]{$A$}--cycle;
     \end{tikzpicture}
     \end{flushright}
     \vspace{3cm}

   \begin{multicols}{2}



'A translation maps triangle $CAT$ onto triangle $DOG$.] \\vspace{0.5cm}\n      \\begin{tikzpicture}[scale=0.9]\n        \\coordinate [label=above left:$C$](A) at (95:2);\n        \\coordinate [label=below:$A$](B) at (0, 0);\n        \\coordinate [label=right:$T$](C) at (-10:3.5);\n        \\draw [thick] (A)--(B)--(C)--cycle;\n  \n        \\draw [thick, xshift=3cm, yshift=1.5cm] (95:2) node[above]{$D$}--\n        (0,0) node[below]{$O$}--\n        (-10:3.5) node[right]{$G$}--cycle;\n      \\end{tikzpicture}\\\\\n      Fill in the blank with the corresponding object.\n      \\begin{enumerate}\n        \\item $A \\rightarrow$ \\rule{2cm}{0.15mm}\n        \\item $\\angle CTA \\cong$ \\rule{2cm}{0.15mm}\n        \\item \\rule{2cm}{0.15mm} $\\cong \\overline {DG}$\n      \\end{enumerate}\n    \\end{multicols}  \\vspace{4cm}\n\n'

In [88]:
problem_df.groupby('problem_set_ID').question.count().head(30)

problem_set_ID
0.0      5
1.0      6
2.0      7
3.0      5
4.0     10
5.0     10
6.0      7
7.0     14
8.0      5
9.0      5
10.0    20
12.0     6
13.0     7
14.0    83
15.0     1
16.0     4
17.0     6
18.0     6
19.0     5
20.0     6
21.0     5
22.0     6
23.0     4
24.0     5
25.0     7
26.0     5
27.0     6
28.0     9
29.0     3
30.0     5
Name: question, dtype: int64

## Printing Problem Sets

In [84]:
title = ('file is 1-6DN-Angle-pairs.tex', '17 August 2020', 'Test run of problems df')
df = problem_df[problem_df.file == '1-6DN_Angle-pairs.tex'].sort_index()
print_problems_df(df, filename='1-6DN_', title=title)

In [134]:
title = ('file is 5-4DN-similarity-ratios.tex, PSet 62', '18 August 2020', 'Issue with multicols')
df = problem_df[problem_df.problem_set_ID == 62].sort_index()
print_problems_df(df, filename='5-4DN-mulitcols-issue', title=title)

In [95]:
print_list = [0, 1, 2]
for pset in print_list:
    title = ('file is 1-temp-' + str(pset), '17 August 2020', 'Test run of print unit')
    df = problem_df[problem_df.problem_set_ID == pset].sort_index()
    print_problems_df(df, filename='1-temp-' + str(pset), title=title)

In [6]:
course_files, course_file_df = map_course_files(course_dir='/Users/chris/GitHub/course-files/Geom')
print('dict len, type: ', len(course_files), type(course_files), '\n df len: ', len(course_file_df), '\n')
course_file_df.unit.unique()

dict len, type:  14 <class 'dict'> 
 df len:  209 



array(['00-Prior-knowledge', '01-Intro', '02-Midpoint+distance',
       '03-Volume+angle-bisectors', '04-Transversals',
       '05-Transformations', '06-Analytic-geometry', '07-Similarity',
       '08-Area+volume', '09-Congruence-transformations',
       '10-Trig+similarity+analytics', '11-Algebra2-intro', 'Slides',
       'Trajectories'], dtype=object)

In [9]:
drop_units = ['00-Prior-knowledge', '11-Algebra2-intro', 'Slides',
       'Trajectories']
for unit in drop_units:
    course_file_df.drop(course_file_df[course_file_df.unit == unit].index, inplace=True)
course_file_df

,unit,file_count,filename
1,01-Intro,15,1-1CW_Measurement.tex
2,01-Intro,15,1-1DN_Algebra.tex
3,01-Intro,15,1-1HW_Algebra.tex
4,01-Intro,15,1-3DN_Function-intercepts.tex
5,01-Intro,15,1-3HW_Equilateral-triangle.tex
...,...,...,...
168,10-Trig+similarity+analytics,7,10-2DN_Analytics-review.tex
169,10-Trig+similarity+analytics,7,10-2bDN_Analytics-review.tex
170,10-Trig+similarity+analytics,7,10-4DN_Tangent-situations.tex
171,10-Trig+similarity+analytics,7,10-4bDN_Analytics-review.tex


In [10]:
pset_df = parse_course_files(course_file_df, course_dir='/Users/chris/GitHub/course-files/Geom')
pset_df[['path', 'unit', 'file']] = pset_df.filename.str.rsplit('/', n=2, expand=True)
pset_df

my IndexError: pop from empty list /Users/chris/GitHub/course-files/Geom/01-Intro/1-7Exam-corrections.tex
Tried to run parse_body on empty file


,filename,head,body,problems_list,problem_count
problem_set_ID,,,,,
0,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[\begin{document}\n, \subsubsection*{1-1 Class...","[\n, \item Given the line segment $\overline...",[Given the line segment $\overline{PQ}$ shown ...,9
1,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[\begin{document}\n, \subsubsection*{1-1 Do No...","[\n, \n, \item In the following two problems...","[\n\n, In the following two problems, solve fo...",8
2,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[\begin{document}\n, \subsubsection*{1-1 Homew...","[\n, \n, \item In the following two problems...","[\n\n, In the following two problems, solve fo...",9
3,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[\begin{document}\n, \subsubsection*{1-3 Do No...","[ \item I have a compass, ruler, protractor, ...","[I have a compass, ruler, protractor, notebook...",6
4,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[\begin{document}\n, \subsubsection*{Homework ...","[\n, \item Construct an equilateral triang...",[Construct an equilateral triangle with $\over...,11
...,...,...,...,...,...
167,/Users/chris/GitHub/course-files/Geom/10-Trig+...,"[\begin{document}\n, \subsubsection*{10.3 Do N...","[\n, \n, \item Write down the slope perpendi...","[\n\n, Write down the slope perpendicular to t...",10
168,/Users/chris/GitHub/course-files/Geom/10-Trig+...,"[\begin{document}\n, \subsubsection*{10.3b Do ...","[\n, \item Write down the slope parallel or ...",[Write down the slope parallel or perpendicula...,13
169,/Users/chris/GitHub/course-files/Geom/10-Trig+...,"[\begin{document}\n, \subsubsection*{10.4 Do N...","[\n, \item Write down the slope perpendicula...",[Write down the slope perpendicular to the giv...,8


In [3]:
from main import parse_problem_sets

Loaded dataframes: 
worksheet_files_df:
 index name:  None 
columns:  Index(['Unnamed: 0', 'unit', 'file_count', 'filename'], dtype='object') 
149 rows:  149 

problem_sets_df:
 index name:  problem_set_ID 
columns:  Index(['filename', 'head', 'body', 'problems_list', 'problem_count',
       'problem_IDs'],
      dtype='object') 
149 rows:  149 

problems_df:
 index name:  problem_ID 
columns:  Index(['problem_set_ID', 'question'], dtype='object') 
1855 rows:  1855 

standards_df:
 index name:  None 
columns:  Index(['course', 'chapter', 'topic', 'ccss_ID'], dtype='object') 
314 rows:  314 

standards_desc_df:
 index name:  ccss_ID 
columns:  Index(['description'], dtype='object') 
131 rows:  131 



In [43]:
p_df = parse_problem_sets(pset_df)

In [44]:
p_df['file'] = p_df.problem_set_ID.apply(lambda x: pset_df.loc[x, 'file'])
p_df['q_len'] = p_df.question.str.len()
p_df.sort_values('q_len', inplace=True)

In [45]:
p_df

,problem_set_ID,question,file,q_len
problem_ID,,,,
2383,155.0,\n,9-4CW_Triangle-congruence-proof.tex,2
1305,82.0,\n,6-3CW_Absense-packet.tex,2
874,61.0,\n\n,5-3DN_area-units.tex,2
264,20.0,\n\n,2-4HW_Parameter-solving.tex,2
291,24.0,\n\n,2-6HW_Compound-shapes.tex,2
...,...,...,...,...
2633,167.0,Mark the missing labels for a reflection acros...,10-2DN_Analytics-review.tex,1382
2664,170.0,Mark the missing labels for a rotation of $180...,10-4bDN_Analytics-review.tex,1382
2643,168.0,Mark the missing labels for a reflection acros...,10-2bDN_Analytics-review.tex,1382


In [46]:
p_df[p_df.problem_set_ID == 82].sort_index()

,problem_set_ID,question,file,q_len
problem_ID,,,,
1273,82.0,"Given $\overline{ABC}$, with $AB=2x-1$, $BC=3x...",6-3CW_Absense-packet.tex,441
1274,82.0,Given $m\angle 3 = x+30$ and $m\angle 5 = 4x-2...,6-3CW_Absense-packet.tex,579
1275,82.0,In the diagram below $m\angle AOB = 6x+5$ and ...,6-3CW_Absense-packet.tex,797
1276,82.0,The point $K$ is the midpoint of $\overline{JL...,6-3CW_Absense-packet.tex,453
1277,82.0,\newpage\n,6-3CW_Absense-packet.tex,9
1278,82.0,"On the graph below, $\overleftrightarrow{AB}$ ...",6-3CW_Absense-packet.tex,951
1279,82.0,\newpage\n,6-3CW_Absense-packet.tex,9
1280,82.0,\n,6-3CW_Absense-packet.tex,3
1281,82.0,Triangle $ABC$ is dilated with a scale factor ...,6-3CW_Absense-packet.tex,669


In [28]:
body = pset_df.loc[2, 'body']
print(type(body))
print(body)

<class 'list'>
['\n', '\n', '  \\item In the following two problems, solve for the value of $x$.\n', '  \\begin{multicols}{2}\n', '    \\begin{enumerate}\n', '      \\item   $x-5=12$ \\vspace{6cm}\n', '      \\item   $13-x=-3$ \\vspace{6cm}\n', '    \\end{enumerate}\n', '  \\end{multicols}\n', '    \\vspace{3cm}\n', '\n', '  \\item Given $g(x)=x^2-5x+15$. Simplify $g(0)$. \\vspace{3cm}\n', '  \\item Given $f(x)=3x-2$. Solve for $x$ such that for $f(x)=13$. \\vspace{4cm}\n', '  \\item Given $h(x)=x^2-x-12$. Solve $h(x)=0$. \\vspace{5cm}\n', '  \n', '\n', '  \\item Simplify each expression. (Leave it in radical form if necessary, not a decimal.)\n', '    \\begin{enumerate}\n', '      \\begin{multicols}{2}\n', '      \\item   $\\sqrt{25}$ \\vspace{1.5cm}\n', '      \\item   $\\sqrt{24}$ \\vspace{1.5cm}\n', '      \\end{multicols}\n', '    \\end{enumerate}\n', '    \\vspace{0.5cm}\n', '\n', '\\newpage\n', '\n', '\\item The line $l$ has the equation $y=\\frac{3}{2}x-1$. \n', '\\begin{enumer

In [28]:
p_df.loc[1240].question #561, 1240

'  \\newpage\n'

In [36]:
problem_df.drop(problem_df.iloc[:57].index, inplace=True) #careful to check rows before deleting (only once)

,problem_set_ID,question,q_len,file
problem_ID,,,,
884,62.0,\newpage\n,9,5-4DN-similarity-ratios.tex
1621,109.0,\newpage\n,9,7-4bCW_Tangent.tex
876,61.0,\newpage\n,9,5-3DN_area-units.tex
1606,107.0,\newpage\n,9,7-3HW_AA-similarity.tex
2531,162.0,\newpage\n,9,9-8bDN-Similarity-review.tex


In [73]:
newpage_df = problem_df[problem_df.q_len < 13].copy()
problem_df = problem_df[problem_df.q_len > 12]
#newpage_df = newpage_df.append(problem_df[problem_df.q_len == 10])

In [77]:
len13 = problem_df[problem_df.q_len == 13].index
print(len13)

Int64Index([1221, 2578,  334,  121,  282,  288, 1904,  316, 1750, 1930,  295,
             329,  160,  430,   67,  452, 2541,   89,  368,   95],
           dtype='int64', name='problem_ID')


In [80]:
#newpage_df
newpage_df = newpage_df.append(problem_df.loc[[1221, 2578, 121, 282, 288, 1904, 316, 1750, 1930, 295, 329,  160,  430,   67,  452, 2541, 89, 95]])
problem_df.drop([1221, 2578, 121, 282, 288, 1904, 316, 1750, 1930, 295, 329,  160,  430,   67,  452, 2541, 89, 95], inplace=True)
newpage_df = newpage_df.append(problem_df.loc[[1959, 1900, 102, 76, 107, 73]])
problem_df.drop([1959, 1900, 102, 76, 107, 73], inplace=True)

,problem_set_ID,question,q_len,file
problem_ID,,,,
929,68.0,\newpage\n,12,5-8DNb-similarity-ratios.tex
917,67.0,\newpage\n,12,5-8DN-similarity-ratios.tex
1221,79.0,\newpage\n,13,6-12Exam-Analytic-geometry.tex
2578,164.0,\newpage \n,13,9-9bExam-transformations.tex
121,12.0,\newpage\n,13,1-7Exam-spicy.tex
282,22.0,\newpage\n,13,2-5HW_Perimeter.tex
288,23.0,\newpage\n,13,2-6DN_Distance+Perimeter.tex
1904,129.0,\newpage \n,13,8-1DN_Circle-trajectories.tex
316,27.0,\newpage\n,13,2-7DN_Compound-areas+perimeters.tex


In [87]:
for q in problem_df[problem_df.file == '9-1DN_Correspondence.tex'].sort_index().question:
    print(q)

Translate $\triangle ABC$ by $(x,y) \rightarrow (x+3, y-5)$. Label the image $\triangle A'B'C'$.
  \begin{center}
      \begin{tikzpicture}[scale=.48]
      \draw [help lines] (-7,-5) grid (10,6);
      \draw [thick, <->] (-7.4,0) -- (10.4,0) node [right] {$x$};
      \draw [thick, <->] (0,-5.4)--(0,6.4) node [above] {$y$};  
      \draw [thick]
        (-4,4) node[below left] {$A$}--
        (3,4) node[above] {$B$}--
        (1,1) node[below] {$C$}--cycle;  
    \end{tikzpicture}
  \end{center}

  
  \begin{multicols}{2}

What transformation maps $\triangle ABC$ onto $\triangle DEC$, shown below? Fully specify the transformation. Complete the table of mappings to  corresponding objects.]  \vspace{0.5cm}
    \begin{enumerate}
      \item $A \rightarrow$ \rule{2cm}{0.15mm}
      \item $B \rightarrow$ \rule{2cm}{0.15mm}
      \item $C \rightarrow$ \rule{2cm}{0.15mm}
      \item $\angle ACB \cong$ \rule{2cm}{0.15mm}
      \item \rule{2cm}{0.15mm} $\cong \overline {DE}$
    \end{enumerate}

In [270]:
p_df = problems_df.copy()
p_df['n_begin_enumerate'] = problems_df.question.str.count(r'\\begin{enumerate}')
p_df['n_end_enumerate'] = problems_df.question.str.count('end{enumerate}')
p_df['n_sub-subsection'] = problems_df.question.str.count(r'\\subsubsection')
p_df['n_subsection'] = problems_df.question.str.count(r'\\subsection')
p_df.sort_values('n_sub-subsection', ascending=False)[:15]

,problem_set_ID,question,n_begin_enumerate,n_end_enumerate,n_sub-subsection,n_subsection
problem_ID,,,,,,
1754,140,\n\subsubsection*{Draw a linear pair with give...,5,4,3,0
1785,142,"\subsubsection*{Perpendicular (bisector, throu...",0,0,2,0
1815,144,\n\begin{enumerate}\n\subsubsection*{Segment a...,2,1,2,0
964,91,\fancyhead[L]{BECA / Dr. Huson / 10th Grade G...,2,1,1,0
356,24,\subsubsection*{Circle the appropriate equatio...,0,0,1,0
111,3,\subsubsection*{Do Now 1b.6: Angle pair practi...,2,1,1,0
1442,120,\setcounter{page}{1}\n%\subsubsection*{13-1 Ho...,2,1,1,0
1631,133,\setcounter{page}{1}\n\subsubsection*{13.9 Exi...,1,0,1,0
1759,140,"\emph{variation} Given two perpendicular rays,...",0,0,1,0
